<a href="https://colab.research.google.com/github/jrivest2/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/Justin_Rivest_DS_module_project_223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 1*

---

In [ ]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

# Decision Trees

## Kaggle

**Task 1:** [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. Go to our Kaggle InClass competition website (the URL is in Slack). Go to the Rules page. Accept the rules of the competition. Notice that the Rules page also has instructions for the Submission process. The Data page has feature definitions.

## Wrangle Data

**Task 2:** Add to the code below so that `id` is set as the index for `df`.

In [2]:
import pandas as pd

df = pd.merge(pd.read_csv(DATA_PATH + 'waterpumps/train_features.csv'),
              pd.read_csv(DATA_PATH + 'waterpumps/train_labels.csv'))
# YOUR CODE HERE
df = df.set_index('id')


**Test 2**

In [3]:
'''Task 2 Testing'''

assert isinstance(df, pd.DataFrame), 'Have you created the DataFrame `df`?'
assert df.shape == (59400, 40), '`df` is the wrong shape. Did you set the index to `id`?'
assert 69572 in df.index, 'The index for `df` is has the wrong values. Did you set the index to `id`?'

## Split Data

**Task 3:** Create your target vector `y` and feature matrix `X`.

In [4]:
target = 'status_group'

# YOUR CODE HERE
X = df.drop(columns=target)
y = df[target]

**Test 3**

In [5]:
'''Task 3 Testing'''

assert isinstance(X, pd.DataFrame), '`X` is the wrong data type.'
assert isinstance(y, pd.Series), '`y` is the wrong data type.'
assert y.shape == (59400,), '`y` is the wrong shape.'
assert X.shape == (59400,39), '`X` is the wrong shape.'

We already have a test set for this model, the `test.csv` that you'll use to make the predictions you'll upload to Kaggle. However, since our competition only allows for 2 submissions per day, we need a way to estimate our training error so that we can quickly iterate and improve our model. We can achieve this by creating a validation set from the data we have.

**Task 4:** Split `X` and `y` into training and validation sets. Your validation set should be 20% of the data you have. You should have four variables: `X_train`, `X_val`, `y_train`, and `y_val`. 

In [6]:
from sklearn.model_selection import train_test_split

# YOUR CODE HERE
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)

**Test 4**

In [7]:
'''Task 4 Testing'''

assert X_train.shape == (47520, 39), '`X_train` is the wrong shape.'
assert X_val.shape == (11880, 39), '`X_val` is the wrong shape.'
assert y_train.shape == (47520,), '`y_train` is the wrong shape.'
assert y_val.shape == (11880,), '`y_val` is the wrong shape.'

## Establish Baseline

**Task 5:** This is a **classification** problem, so you need to establish the baseline accuracy for your training set. Find the majority class for `y_train` and calculate the percentage of labels in `y_train` belonging to that class. Assign your answer to the variable name `baseline_acc`.

In [8]:
# YOUR CODE HERE
baseline_acc = y_train.value_counts(normalize=True).max()

# print('Baseline Accuracy:', baseline_acc)

**Task 5**

In [9]:
'''Task 5 Testing'''

assert isinstance(baseline_acc, float), '`baseline_acc` should be a `float`.'
assert 0.0 <= baseline_acc <= 1.0, '`baseline_acc` is a score that should be between 0 and 1.'

## Build Model

**Task 6:** Create a model named `model` and train it with your training data. Your model should be a pipeline with (a) transformers that you think are appropriate to this dataset and (b) a `DecisionTreeClassifier` as your predictor. **Tip:** How can you transform categorical features and missing values in order to train your model?

In [10]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

# YOUR CODE HERE
model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    DecisionTreeClassifier(random_state=42,max_depth=14,)
)

model.fit(X_train, y_train);

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Test 6**

In [11]:
'''Task 6 Testing'''

assert len(model) > 1, 'Your model pipeline should have multiple steps.'
assert isinstance(model[-1], DecisionTreeClassifier), 'Your pipline should end in a `DecisionTreeClassifier`.'
assert hasattr(model, 'classes_'), 'Have you fit your model?'

## Check Metrics

**Task 7:** Calculate the training and validation accuracy of your model, and assign them to the valiables `training_acc` and `validation_acc`, respectively. Your validation accuracy should be greater than your baseline accuracy.

In [12]:
# YOUR CODE HERE
training_acc = model.score(X_train, y_train)
validation_acc = model.score(X_val, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.8422558922558923
Validation Accuracy: 0.7605218855218855


**Test 7**

In [13]:
'''Task 7 Testing'''

assert isinstance(training_acc, float)
assert isinstance(validation_acc, float)
assert 0.0 <= training_acc <= 1.0
assert 0.0 <= validation_acc <= 1.0

## Create Kaggle Submission

**Task 8:** Load `'waterpumps/test.csv'` into a DataFrame named `X_test`. Generate a list of predictions, and then put them into a DataFrame `submission`. Be sure that `submission` has the same index as `X_test` and that the column name for your predictions is `'status_group'`.  

In [14]:
X_test = pd.read_csv(DATA_PATH + 'waterpumps/test_features.csv', index_col='id')
# YOUR CODE HERE
y_predict = model.predict(X_test)
submission = pd.DataFrame(y_predict, columns=['status_group'],index=X_test.index)
 



**Test 8**

In [15]:
'''Task 8 Testing'''

assert isinstance(submission, pd.DataFrame), '`submission` should be a DataFrame.'
assert len(submission) == 14358, '`submission` should have 14358 rows.'

**Task 9 (`stretch goal`):** Save `submission` as a csv file using [`.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) and submit it to our Kaggle competition.

In [16]:
# YOUR CODE HERE
submission.to_csv('justin_rivest_submit_02.csv')

## Explain

**Task 10 (`stretch goal`):** Make a horizontal barchart of the 10 most important features for your model.

In [17]:
import matplotlib.pyplot as plt

# YOUR CODE HERE


In [18]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(47520, 39)
(11880, 39)
(47520,)
(11880,)


In [29]:
from sklearn.ensemble import RandomForestClassifier

model_forest = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42,
                           max_depth=21,
                           max_samples=0.7,
                           criterion='gini',
                           n_estimators=200,
                           n_jobs=-1
                           )
)

model_forest.fit(X_train, y_train);

In [30]:
training_acc = model_forest.score(X_train, y_train)
validation_acc = model_forest.score(X_val, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.9595328282828283
Validation Accuracy: 0.8154040404040404


In [21]:
y_predict = model_forest.predict(X_test)
submission = pd.DataFrame(y_predict, columns=['status_group'],index=X_test.index)
 


In [22]:
submission.to_csv('justin_rivest_submit_03.csv')

In [31]:
# Reset data to not include validation set because validating will be taken 
#  care of by the cross-validation

X_train = df.drop(columns=target)
y_train = df[target]

In [37]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
# Hyper tuning 
tuned = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42,
                           criterion='gini',
                           )
)

params = {
    'randomforestclassifier__n_estimators' : range(50,251,50),
    'randomforestclassifier__max_depth' : range(5,31,5),
    'randomforestclassifier__max_samples' : np.arange(0.2,0.7,0.2),
    'simpleimputer__strategy' : ['mean','most_frequent','median']
}
model_tuned = GridSearchCV(
    tuned,
    param_grid=params,
    cv=5,
    scoring='accuracy',
    return_train_score=True,
    verbose=1,
    n_jobs=-1
)
model_tuned.fit(X_train, y_train);

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 42.8min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 90.3min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 165.2min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed: 188.9min finished


In [38]:
model_tuned.best_params_

{'randomforestclassifier__max_depth': 25,
 'randomforestclassifier__max_samples': 0.6000000000000001,
 'randomforestclassifier__n_estimators': 250,
 'simpleimputer__strategy': 'mean'}

In [39]:
model_tuned.best_score_

0.8123905723905723

In [40]:
training_acc = model_tuned.score(X_train, y_train)
validation_acc = model_tuned.score(X_val, y_val)

print('Training Accuracy:', training_acc)
print('Validation Accuracy:', validation_acc)

Training Accuracy: 0.9706902356902357
Validation Accuracy: 0.9703703703703703


In [41]:
y_predict = model_tuned.predict(X_test)
submission = pd.DataFrame(y_predict, columns=['status_group'],index=X_test.index)
 


In [42]:
submission.to_csv('justin_rivest_submit_04.csv')